<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1s0UmeBOolMe81jdfj7Pt4aPnRLmS8jIA#scrollTo=eUECVlnNO2Gz"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/jama5262/tensorflow_emotion_text_detector"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


### Import Tensorflow and Pandas
---



In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from absl import logging

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf
import pandas as pd

tf.__version__

'1.15.0'

### Fetching our dataset from Google drive

**You can skip this if you have downloaded the dataset**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load the `emotion.csv` file to pandas

In [0]:
df = pd.read_csv('/content/drive/My Drive/ML/csv files/emotion.csv')
df

,id,text,emotions
0,27383,i feel awful about it too because it s my job ...,sadness
1,110083,im alone i feel awful,sadness
2,140764,ive probably mentioned this before but i reall...,joy
3,100071,i was feeling a little low few days back,sadness
4,2837,i beleive that i am much more sensitive to oth...,love
...,...,...,...
416804,566,that was what i felt when i was finally accept...,joy
416805,36236,i take every day as it comes i m just focussin...,fear
416806,76229,i just suddenly feel that everything was fake,sadness
416807,131640,im feeling more eager than ever to claw back w...,joy


### Drop the `id` column and any `NaN` rows within the dataframe

In [0]:
df = df.drop("id", axis=1)
df = df.dropna()

### The below code will output all the emotion `labels` available in the dataset
And these are `['fear', 'surprise', 'sadness', 'anger', 'joy', 'love']`

In [0]:
emotions = list(set(list(df.emotions)))
emotions

['anger', 'fear', 'sadness', 'surprise', 'joy', 'love']

### Next we are going to check how many, data are available for each of the emotion labels

It turns out that the `joy` label is more biased

In [0]:
countEachEmotion = df.groupby(["emotions"]).size()
countEachEmotion

emotions
anger        57317
fear         47712
joy         141067
love         34554
sadness     121187
surprise     14972
dtype: int64

### We will be factorizing the label to int formats
e.g sadness will be given 0, joy will be given 1 and so on

In [0]:
df.emotions, uniques = pd.factorize(df.emotions)
uniques

Index(['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], dtype='object')

### Import the Tenforflow hub and sklearn train_test_split

In [0]:
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

### Next we will get our trainig and test datasets

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df, df.emotions, test_size=0.20)

### Setting our estimator `input_fn` on the training dataset

In [0]:
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    X_train, y_train, num_epochs=None, shuffle=True)

### We next set our `feature_column` and load our tf hub text embedding

In [0]:
embedded_text_feature_column = hub.text_embedding_column(
    key="text", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")

### Setting up the th estimator model (DNNClassifier)



In [0]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[100, 50],
    feature_columns=[embedded_text_feature_column],
    n_classes=6,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001))

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpemj02fmv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f983e575c50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpemj02fmv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f983e575c50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### We then train our model

In [0]:
estimator.train(input_fn=train_input_fn, steps=5000)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:loss = 111.05366, step = 4901 (0.551 sec)


INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmpemj02fmv/model.ckpt.


INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmpemj02fmv/model.ckpt.


INFO:tensorflow:Loss for final step: 137.23575.


INFO:tensorflow:Loss for final step: 137.23575.


### Set up our input functions to evaluate our model

In [0]:
predict_train_input_fn = tf.estimator.inputs.pandas_input_fn(
    X_train, y_train, shuffle=False)

predict_test_input_fn = tf.estimator.inputs.pandas_input_fn(
    X_test, y_test, shuffle=False)

### Evaluating our model and output the accuracy

The accuracy outputted are for the training dataset and the test

In [0]:
train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

print("Training set accuracy: {accuracy}".format(**train_eval_result))
print("Test set accuracy: {accuracy}".format(**test_eval_result))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.63556534, average_loss = 0.9496476, global_step = 5000, loss = 121.417984


INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.63556534, average_loss = 0.9496476, global_step = 5000, loss = 121.417984


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpemj02fmv/model.ckpt-5000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpemj02fmv/model.ckpt-5000


Training set accuracy: 0.6395918726921082
Test set accuracy: 0.6355653405189514


### Import numpy

In [0]:
import numpy as np

### Setup functions to sort the probability in decending order and add map the labels to their respective probabilities

In [0]:
def sortByProb(prob):
  sorted_x = sorted(prob.items(), key=lambda x: x[1], reverse=True)
  return sorted_x

def showProb(prob):
  return sortByProb({ uniques[i] : prob[i] for i in range(0, len(prob)) })

### We then test our model with sample data

In [0]:
raw_test = [
    "So happy to see you",
]

predict_input_fn = tf.estimator.inputs.numpy_input_fn({"text": np.array(raw_test).astype(np.str)}, shuffle=False)
results = estimator.predict(predict_input_fn)
print(showProb(next(results)["probabilities"]))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


[('joy', 0.9136373), ('surprise', 0.079023294), ('love', 0.003275134), ('anger', 0.002794646), ('sadness', 0.001097586), ('fear', 0.00017200041)]


### We export our model

In [0]:
feature_spec = tf.feature_column.make_parse_example_spec([embedded_text_feature_column])

serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

estimator.export_saved_model("emotion_text_model", serving_input_fn)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Assets written to: emotion_text_model/temp-b'1573644768'/assets


INFO:tensorflow:Assets written to: emotion_text_model/temp-b'1573644768'/assets


INFO:tensorflow:SavedModel written to: emotion_text_model/temp-b'1573644768'/saved_model.pb


INFO:tensorflow:SavedModel written to: emotion_text_model/temp-b'1573644768'/saved_model.pb


b'emotion_text_model/1573644768'

### Load the exported model and run with another sample dataset

In [0]:
from tensorflow.contrib import predictor

# Remember to replace the direcory with your exported model
predict_fn = predictor.from_saved_model("/content/emotion_text_model/1573644768")

test_data = b'I hate you'

test = {
    "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[test_data]))
}

example = tf.train.Example(
    features=tf.train.Features(
        feature=test
    )
)
serialized_example = example.SerializeToString()

results = predict_fn({'inputs': [serialized_example]})
print(createDict(results["scores"][0]))

INFO:tensorflow:Restoring parameters from /content/emotion_text_model/1573639213/variables/variables


INFO:tensorflow:Restoring parameters from /content/emotion_text_model/1573639213/variables/variables


[('anger', 0.4960261), ('sadness', 0.48368546), ('love', 0.010394456), ('fear', 0.007681031), ('surprise', 0.001107882), ('joy', 0.0011050951)]
